

*   Unsupervised pretraining
*   Supervised finetuning
* RLHF (Direct Preference Optimization and Proximal Preference Optimization)

We will Finetune LLM today Mistral, LLAMA, Deepseek.
WE have billions of parameter in these model and we cannot perform supervised fine tuning.
WE will perform PEFT.Parameter efficient finetuning. It takes the subset of trainable parameters from the pretrained model. The attention layer in the transformers have trainable parameters.  which will be used to finetune the weights. LORA will be used.
Adaptors are used to freeze the model weights and allow trainable parameters to be tranined

Quantization + LORA. we reduce the precision of the weights. from FP16 to FP8 or FP4.

Different quantization techniques

- GGML
- GGUF
- GPTQ




In [2]:
! pip install accelerate peft bitsandbytes trl py7zr auto-gptq optimum transformers

  Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl.metadata (5.8 kB)
  Using cached trl-0.15.1-py3-none-any.whl.metadata (11 kB)
  Using cached py7zr-0.22.0-py3-none-any.whl.metadata (16 kB)
  Using cached auto_gptq-0.7.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached optimum-1.24.0-py3-none-any.whl.metadata (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 10.2 M

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import torch
import os
from datasets import load_dataset,Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,GPTQConfig
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

In [5]:
data= load_dataset("samsum", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [6]:
data_df=data.to_pandas()

In [7]:
data_df.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [8]:
data_df["text"]=data_df[["dialogue","summary"]].apply(lambda x:"###Human: Summarize this following dialogue: " + x["dialogue"] + "\n###Assistant: " +x["summary"] ,axis=1)

In [13]:
data_df[["text"]]

,text
0,###Human: Summarize this following dialogue: A...
1,###Human: Summarize this following dialogue: O...
2,###Human: Summarize this following dialogue: T...
3,###Human: Summarize this following dialogue: E...
4,###Human: Summarize this following dialogue: S...
...,...
14727,###Human: Summarize this following dialogue: R...
14728,###Human: Summarize this following dialogue: T...
14729,###Human: Summarize this following dialogue: J...
14730,###Human: Summarize this following dialogue: J...


In [14]:
data= Dataset.from_pandas(data_df)

In [24]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [26]:
tokenizer.pad_token=tokenizer.eos_token

In [15]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,    # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,  # Use fp16 for computation
    bnb_4bit_use_double_quant=True,  # Use double quantization for memory efficiency
)

In [18]:
model=AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    quantization_config=quantization_config,
    device_map="auto"
                                     )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
model=prepare_model_for_kbit_training(model)

LORA

In [20]:
peft_config=LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
)

In [21]:
model=get_peft_model(model,peft_config=peft_config)

In [22]:
traning_arguments = TrainingArguments(
    output_dir="mistral-finetuned-samsum",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=100,
    num_train_epochs=1,
    fp16=True,
    push_to_hub=True,
    report_to="none"
)

In [28]:
trainer=SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=traning_arguments,
)

<ipython-input-28-42036fefe4a3>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer=SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/14732 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

In [30]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 76.12 MiB is free. Process 45125 has 14.66 GiB memory in use. Of the allocated memory 13.70 GiB is allocated by PyTorch, and 845.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)